In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px

from plotly.subplots import make_subplots

In [17]:
csv_dir = Path("./runs")
csv_dir_leg = Path("./runs/legacy")

csv_files = list(csv_dir.glob("*.csv"))
csv_files_leg = list(csv_dir_leg.glob("*.csv"))

files = csv_files + csv_files_leg

print(files)
dataframes = []
for file in files:
    name = str(file).split("/")[-1].split(".")[0]
    df = pd.read_csv(file)
    df['id'] = name
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

df['reward_100ma'] = df.groupby('id')['reward'].transform(lambda x: x.rolling(window=100).mean())
df['cumls_steps'] = df.groupby('id')['steps'].transform(lambda x: x.cumsum())
df['cumls_time'] = df.groupby('id')['time_seconds'].transform(lambda x: x.cumsum())
df = df.dropna()
data = df

df

[PosixPath('runs/dqn_double_opt_v3_3k.csv'), PosixPath('runs/ppo_config1.csv'), PosixPath('runs/dqn_duel_v3_3k.csv'), PosixPath('runs/dqn_duel_opt_v3_3k.csv'), PosixPath('runs/ppo_config2.csv'), PosixPath('runs/ppo_config3.csv'), PosixPath('runs/dqn_opt_v3_3k.csv'), PosixPath('runs/dnq_opt_6k.csv'), PosixPath('runs/dqn_double_v3_3k.csv'), PosixPath('runs/dqn_v3_3k.csv'), PosixPath('runs/dqn_double_opt_6k.csv'), PosixPath('runs/legacy/dqn.csv'), PosixPath('runs/legacy/dqn_double.csv'), PosixPath('runs/legacy/dqn_duel.csv'), PosixPath('runs/legacy/dqn_double_opt.csv'), PosixPath('runs/legacy/dqn_duel_opt.csv'), PosixPath('runs/legacy/dqn_opt.csv')]


,episode,reward,time_seconds,steps,id,reward_100ma,cumls_steps,cumls_time
99,100,-83.322901,0.513232,1000,dqn_double_opt_v3_3k,-157.794478,60265,31.638080
100,101,-66.435506,0.499271,1000,dqn_double_opt_v3_3k,-157.788338,61265,32.137351
101,102,-114.029876,0.562422,1000,dqn_double_opt_v3_3k,-155.382307,62265,32.699773
102,103,-34.388386,0.554285,1000,dqn_double_opt_v3_3k,-151.356434,63265,33.254058
103,104,-79.903947,0.569125,1000,dqn_double_opt_v3_3k,-128.551320,64265,33.823183
...,...,...,...,...,...,...,...,...
56995,2996,265.031030,0.544449,212,dqn_opt,257.565856,573667,2003.473161
56996,2997,273.154505,0.608823,247,dqn_opt,257.541837,573914,2004.081984
56997,2998,303.107110,0.453290,179,dqn_opt,258.214277,574093,2004.535273
56998,2999,227.975601,0.609553,239,dqn_opt,257.853873,574332,2005.144826


In [18]:
# px.line(df, x='episode', y='reward_100ma', color='id', facet_col='id', facet_col_wrap=3, width=1600, height=800)


In [19]:
df = data.copy()

df = df[~df['id'].str.contains('6k')]

px.line(df, x='episode', y='reward_100ma', color='id', width=1400, height=700)

In [5]:
# px.line(df, x='episode', y='reward_100ma', color='id', facet_col='id', facet_col_wrap=3, width=1600, height=800)

df = df[df['id'].isin(['dqn_v3', 'dqn_opt'])]

final_values = df.groupby('id')['reward_100ma'].last().sort_values(ascending=False)
category_orders = {'id': final_values.index.tolist()}

fig = px.line(df, x='episode', y='reward_100ma', color='id', category_orders=category_orders, width=1300, height=600)

fig

# fig.add_annotation(
#     x=x_max, y=200, text="Human Level", showarrow=False,
#     xanchor="left", xshift=5, yshift=10,  
#     font=dict(color="black", size=12),
#     row=1, col=1
# )

# fig.add_hline(
#     y=200,
#     line=dict(color="black", width=1.5, dash="dash"),
#     row=1, col=1
# )

In [176]:


final_values = df.groupby('id')['reward_100ma'].last().sort_values(ascending=False)
category_orders = {'id': final_values.index.tolist()}

dqn_reward = px.line(df, x='episode', y='reward_100ma', color='id', 
               title="Model Performance Comparison", width=1200, height=600,
               category_orders=category_orders)



In [ ]:
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=("Reward (100ma)", "Cumulative Steps", "Cumulative Time"),
                   vertical_spacing=0.1,
                   column_widths=[0.5, 0.5],
                   row_heights=[0.65, 0.35],  
                   specs=[[{"colspan": 2}, None], [{}, {}]])

# Fix the typo in the filter: 'dnq' should be 'dqn'
fig_ppo_reward = px.line(df[df.id.str.contains('dqn')], x='episode', y='reward_100ma', color='id', title='DQN Reward (100ma)', width=1000, height=500)
fig_ppo_cumls_steps = px.line(df[df.id.str.contains('dqn')], x='episode', y='cumls_steps', color='id', title='DQN Cumulative Steps', width=1000, height=500)
fig_ppo_cumls_time = px.line(df[df.id.str.contains('dqn')], x='episode', y='cumls_time', color='id', title='DQN Cumulative Time', width=1000, height=500)

used_names = set()  
for trace in dqn_reward.data:
    if trace.name in used_names:
        trace.showlegend = False
    else:
        used_names.add(trace.name)
    fig.add_trace(trace, row=1, col=1)
    
for trace in fig_ppo_cumls_steps.data:
    trace.showlegend = False
    fig.add_trace(trace, row=2, col=1)
    
for trace in fig_ppo_cumls_time.data:
    trace.showlegend = False
    fig.add_trace(trace, row=2, col=2)

# Add reference lines with annotations
x_min = df[df.id.str.contains('dqn')]['episode'].min()
x_max = df[df.id.str.contains('dqn')]['episode'].max()

fig.add_annotation(
    x=x_max, y=200, text="Human Level", showarrow=False,
    xanchor="left", xshift=5, yshift=10,  
    font=dict(color="black", size=12),
    row=1, col=1
)

fig.add_hline(
    y=200,
    line=dict(color="black", width=1.5, dash="dash"),
    row=1, col=1
)

# Update axis titles for better clarity
fig.update_xaxes(title_text="Episode", row=2, col=1)
fig.update_xaxes(title_text="Episode", row=2, col=2)
fig.update_yaxes(title_text="Cumulative Steps", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Time (s)", row=2, col=2)

fig.update_layout(
    height=850,
    width=1300,
    title_text="DQN Performance Report",
    hovermode="x unified", 
    margin=dict(t=80, l=50, r=50, b=50)
)